In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')
dataset_dir = os.path.join(os.path.dirname(dataset),'aclImdb')
os.listdir(dataset_dir)

84125825/84125825 [==============================] - 3s 0us/step


['train', 'README', 'imdb.vocab', 'imdbEr.txt', 'test']

In [3]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['urls_neg.txt',
 'urls_unsup.txt',
 'unsupBow.feat',
 'pos',
 'unsup',
 'urls_pos.txt',
 'labeledBow.feat',
 'neg']

In [4]:
sample_file = os.path.join(train_dir,"pos/1181_9.txt")
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [5]:
remove_dir = os.path.join(train_dir,"unsup")
shutil.rmtree(remove_dir)

In [6]:
batch_size=42
seed=42
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)
print(raw_train_ds.take(1))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
<_TakeDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [7]:
for text_batch ,label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("review",text_batch.numpy()[i])
    print("label",label_batch.numpy()[i])

review b"Rob Estes, Josie Bisset and a crap load of kids that look nothing like either of them.<br /><br />Basically, Rob and Josie have a shotgun wedding on a drunken night during a Vegas vacation. They each come home to find that their respective children already know of the nuptials due to tabloid-like not-so-fodder. They, Rob and Josie, move both of them and their eight kids into one or the other's house.<br /><br />Rob builds furniture, I think, which is close enough to Frank Lambert's (Patrick Duffy) construction job on the much similar Step by Step to warrant eternal mockage.<br /><br />Josie is some sort of cookie-making queen, though it doesn't look like she makes any of the cookies. Not close enough to Carol Foster's (Suzanne Somers)hairdressing job to warrant likeness mockage, but hilariously preposterous enough to warrant atrocity mockage.<br /><br />Unlike Step by Step, they were a couple before the vacation and actually knew one another's last names, or so one assumes if 

In [8]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


In [9]:
raw_validation_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)
for text_batch ,label_batch in raw_validation_ds.take(1):
  for i in range(3):
    print("review",text_batch.numpy()[i])
    print("label",label_batch.numpy()[i])


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
review b"Quite one of the worst films I have ever seen. Terrible acting, laughable 'action' (it's clear that the cars are travelling slowly), atrocious script, hideously unsatisfying ending and incompetent direction make a hash of a movie. We know Judge Reinhold is a fine actor, but he should be ashamed of this detritus. There is no great tension within the car and, when the characters stumble upon moments of hope, they laugh like inane banshees for some reason, even 'high five-ing' when they see the bridge lowered!<br /><br />Also, the chain of events that lead these people to share the same car strains credibility. Apparently based on true events, though? If that's the case, truth is evidently stranger than fiction! Unfortunate then, that it was portrayed in such an inept manner."
label 0
review b'Opening scene \'explains\' why Hurt is later \'immune\' to the \'Contaminated Man\'. Too bad it doesn\'t explain a

In [10]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)
for text_batch ,label_batch in raw_test_ds.take(1):
  for i in range(3):
    print("review",text_batch.numpy()[i])
    print("label",label_batch.numpy()[i])

Found 25000 files belonging to 2 classes.
review b'My wife is a mental health therapist and we watched it from beginning to end. I am the typical man and can not stand chick flicks, but this movie is unbelievable. If you want to see what it is like for someone who is going through these type of struggles, this is the movie for you. As I watched it I found myself feeling sorry for him and others like him. <br /><br />***Spoiler*** Plus the fact that all the individuals in the movie including the people in the mental institution were the actual people in real life made it that more real.<br /><br />A must see for someone in the mental health profession!'
label 1
review b'This version of Mansfield Park, while being extremely accurate to the novel lacked the compassion I felt for Fanny which is crucial and central to this Jane Austen story. This was due to the total lack of acting ability by the actress, Sylvestra Le Touzel. She had no appeal and at times appeared to be either lost in spac

In [11]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [12]:
vectorized_layer = layers.TextVectorization(
    standardize = custom_standardization,
    max_tokens = 10000,
    output_mode = 'int',
    output_sequence_length=255
)

In [13]:
train_text = raw_train_ds.map(lambda x, y: x)
print(train_text.take(1))
vectorized_layer.adapt(train_text)

<_TakeDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>


In [14]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorized_layer(text), label

text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

print("1287 ---> ",vectorized_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorized_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorized_layer.get_vocabulary())))


Review tf.Tensor(b"Live Feed is set in some unnamed Chinese/Japanese Asian district somewhere as five American friends, Sarah (Ashley Schappert), Emily (Taayla Markell), Linda (Caroline Chojnacki), Mike (Lee Tichon) & Darren (Rob Scattergood) are enjoying a night on the town & taking in the sights. After a scuffle in a bar with a Japanese Triad boss (Stephen Chang) they decide to check out a porno theatre, as you would. Inside they are separated & quickly find out that the place belongs to the Triad boss who uses it to torture & kill people for reasons which aren't made clear. Can local boy Miles (Kevan Ohtsji) save them?<br /><br />This Canadian production was co-written, produced & directed by Ryan Nicholson who also gets a prosthetic effects designer credit as well, one has to say that Live Feed is another pretty poor low budget shot on a camcorder type horror film that seems to exist only to cash in on the notoriety & success of Hostel (2005) & the mini craze for 'torture porn' as 

In [16]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_validation_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [20]:
model = tf.keras.Sequential([
    layers.Embedding(10000,16),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1,activation='sigmoid')
]
)

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 16)          160000    
                                                                 
 dropout_3 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160017 (625.07 KB)
Trainable params: 160017 (625.07 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [22]:
model.compile(loss = losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)])

In [24]:
history = model.fit(
         train_ds,
         validation_data=val_ds,
         epochs=10
)

Epoch 1/10
477/477 [==============================] - 9s 15ms/step - loss: 0.6738 - binary_accuracy: 0.6820 - val_loss: 0.6385 - val_binary_accuracy: 0.7670
Epoch 2/10
477/477 [==============================] - 4s 9ms/step - loss: 0.5835 - binary_accuracy: 0.7871 - val_loss: 0.5380 - val_binary_accuracy: 0.8098
Epoch 3/10
477/477 [==============================] - 4s 8ms/step - loss: 0.4866 - binary_accuracy: 0.8299 - val_loss: 0.4585 - val_binary_accuracy: 0.8382
Epoch 4/10
477/477 [==============================] - 5s 10ms/step - loss: 0.4159 - binary_accuracy: 0.8561 - val_loss: 0.4050 - val_binary_accuracy: 0.8512
Epoch 5/10
477/477 [==============================] - 4s 8ms/step - loss: 0.3674 - binary_accuracy: 0.8712 - val_loss: 0.3699 - val_binary_accuracy: 0.8610
Epoch 6/10
477/477 [==============================] - 4s 8ms/step - loss: 0.3344 - binary_accuracy: 0.8815 - val_loss: 0.3462 - val_binary_accuracy: 0.8666
Epoch 7/10
477/477 [==============================] - 5s 10ms/

In [25]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

596/596 [==============================] - 6s 10ms/step - loss: 0.3179 - binary_accuracy: 0.8718
Loss:  0.31793540716171265
Accuracy:  0.8717600107192993


In [26]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy'])

In [28]:
export_model = tf.keras.Sequential([
  vectorized_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

596/596 [==============================] - 4s 7ms/step - loss: 0.5940 - accuracy: 0.5000
0.5
